# Setup

Switch kernel to bw25 if necessary

In [1]:
import bw_processing as bwp
import matrix_utils as mu
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import numpy as np
import seaborn as sb
import pandas as pd
from pathlib import Path

In [129]:
import bw2analyzer as ba

In [4]:
if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")

In [5]:
bd.projects # dont know if we need to create our own project?

Brightway2 projects manager with 9 objects:
	Correlated and dependent sampling
	Lithium GSA
	brightcon22
	datapackage demo
	default
	ei38-teaching
	ei38-teaching-25
	ei_38
	your project name
Use `projects.report()` to get a report on all projects.

In [53]:
bd.projects.set_current('ei38-teaching-25')

In [54]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

In [ ]:
# easy to change names of biosphere and ei db:s
biosphere = 'biosphere3'
ecoinvent = 'ei 3.8 cutoff'

In [ ]:
# could be good to have idk
eidb = bd.Database('ei 3.8 cutoff')
my_bio = bd.Database(biosphere)

In [ ]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

# Template for searching activities etc

In [ ]:
bd.Database(biosphere).search('carbon dioxide')

In [ ]:
bd.Database(ecoinvent).search('copper')

In [31]:
activity_I_want = [act for act in bd.Database(biosphere) if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
         ][0]
activity_I_want

'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))

In [32]:
activity_I_want.key

('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9')

In [34]:
bd.get_activity(activity_I_want.key)

'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))

In [51]:
bd.Database(biosphere).search('carbon dioxide', filter={'categories':'urban', 'name':'fossil'})

Excluding 19 filtered results


['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground'))]

In [ ]:
# Using search
bd.Database(ecoinvent).search('transport', filter={'name':'lorry'})

# Methods

In [39]:
ipcc2013 = [m for m in bd.methods if 'IPCC' in m[0]
                    and ('2013') in str(m)
                    and 'GWP 100' in str(m)
                    and 'no LT' not in str(m)][0]

In [42]:
type(ipcc2013)

tuple

In [40]:
ipcc_2013_method = bd.Method(ipcc2013)

In [41]:
type(ipcc_2013_method)

bw2data.method.Method

In [41]:
ipcc_2013_method.name

('IPCC 2013', 'climate change', 'GWP 100a')

In [42]:
ipcc_2013_method.metadata

{'description': "IPCC characterisation factors for the direct global warming potential of air emissions in ecoinvent 3.2. See the ecoinvent report 'Implementation of IPCC impact assessment method 2007 and 2013 to ecoinvent database 3.2 (2015.11.30)' for more details. Doesn't include: indirect formation of dinitrogen monoxide from nitrogen emissions, radiative forcing due to emissions of NOx, water, sulphate, etc. in the lower stratosphere + upper troposphere, carbon climate feedbacks, and any emissions which have cooling effects (e.g. black carbon). All CO is assumed to convert completely to CO2. Biogenic CO2 uptake and biogenic CO2 emissions are not characterised, except for 'Carbon dioxide, from soil or biomass stock' (deforestation and land transformation).",
 'filename': 'LCIA_Implementation_3.8.xlsx',
 'unit': 'kg CO2-Eq',
 'abbreviation': 'ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf',
 'num_cfs': 211}

In [43]:
ipcc_2013_method.metadata['unit']

'kg CO2-Eq'

# Creating LCI

In [74]:
# creating new activity? let's do that in our database?
bd.databases

Databases dictionary with 5 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	charcrete
	ei 3.8 cutoff

In [73]:
user_db = bd.Database("charcrete")

In [70]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

In [84]:
example_data3 = {
    ("charcrete", "A"): {
        "name": "A",
        "exchanges": [{
            "amount": 1.0,
            "input": ("charcrete", "B"),
            "type": "technosphere"
            }],
        'unit': 'kilogram',
        'location': 'here',
        'categories': ("very", "interesting")
        },
    ("charcrete", "B"): {
        "name": "B",
        "exchanges": [],
        'unit': 'microgram',
        'location': 'there',
        'categories': ('quite', 'boring')
        }
    }

In [85]:
user_db.write(example_data3)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 10/26/2022 12:16:34
  Finished: 10/26/2022 12:16:34
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.20


In [87]:
len(user_db)

2

In [109]:
user_db.search('*')

[]

In [104]:
act = user_db.search('*')[0]

In [107]:
del(act)

In [108]:
user_db.search('*')

[]

In [110]:
user_db

Brightway2 SQLiteBackend: charcrete

In [111]:
len(user_db)

0

## find data

In [112]:
bd.Database(ecoinvent).search('forwarding')

['market for forwarding, forwarder' (hour, GLO, None),
 'forwarding, forwarder' (hour, RER, None),
 'forwarding, forwarder' (hour, RoW, None)]

In [115]:
activity_I_want = [act for act in bd.Database(ecoinvent) if 'forwarding' in act['name'] and act['location'] == 
                   'RER'
         ][0]
activity_I_want

'forwarding, forwarder' (hour, RER, None)

In [116]:
forwarding_act = [act for act in bd.Database(ecoinvent) if 'forwarding' in act['name'] and act['location'] == 
                   'RER'][0]

In [117]:
forwarding_act

'forwarding, forwarder' (hour, RER, None)

In [118]:
bd.Database(ecoinvent).search('wood chipping, mobile chipper, at forest road')

['market for wood chipping, chipper, mobile, diesel, at forest road' (hour, GLO, None),
 'wood chipping, mobile chipper, at forest road' (hour, RER, None),
 'wood chipping, mobile chipper, at forest road' (hour, RoW, None)]

In [119]:
activity_I_want = [act for act in bd.Database(ecoinvent) if 'wood chipping, mobile chipper, at forest road' in act['name'] and act['location'] == 
                   'RER'
         ][0]
activity_I_want

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

In [120]:
activity_I_want

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

In [121]:
wood_chipping_act = [act for act in bd.Database(ecoinvent) if 'wood chipping, mobile chipper, at forest road' in act['name'] 
                     and act['location'] == 'RER'][0]

In [122]:
wood_chipping_act

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

# regionalization of concrete production?

In [134]:
concrete_market = bd.get_activity(database=ecoinvent, name="market group for concrete, normal", location = "GLO")

In [125]:
concrete_market

'market group for concrete, normal' (cubic meter, GLO, None)

In [126]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [131]:
ba.print_recursive_calculation(concrete_market, ipcc, cutoff=0.001) # CA
# short cut: we assume that the ones contributing most is the ones we need to regionalize
# is there a way to sort this data?

Fraction of score | Absolute score | Amount | Activity
0001 | 283.4 |     1 | 'market group for concrete, normal' (cubic meter, GLO, None)
  0.0144 | 4.086 | 0.0226 | 'market for concrete, normal' (cubic meter, BR, None)
    0.0144 | 4.086 | 0.0226 | 'concrete, all types to generic market for concrete, normal strength' 
      0.00275 | 0.7798 | 0.003616 | 'market for concrete, 30MPa' (cubic meter, BR, None)
      0.0115 | 3.253 | 0.01876 | 'market for concrete, 25MPa' (cubic meter, BR, None)
  0.00208 | 0.5893 | 0.002355 | 'market for concrete, normal' (cubic meter, CO, None)
    0.00208 | 0.5893 | 0.002355 | 'concrete, all types to generic market for concrete, normal strength' 
      0.00208 | 0.5893 | 0.002355 | 'market for concrete, 20MPa' (cubic meter, CO, None)
  0.0261 |  7.39 | 0.01907 | 'market for concrete, normal' (cubic meter, IN, None)
    0.0261 |  7.39 | 0.01907 | 'concrete, all types to generic market for concrete, normal strength' 
      0.0261 |  7.39 | 0.01907 | 'mark

In [138]:
# could also do specifically for 20MPa
concrete_markets = [act for act in bd.Database(ecoinvent) if 'market for concrete, 20MPa' in act['name']]

In [139]:
concrete_markets

['market for concrete, 20MPa' (cubic meter, PE, None),
 'market for concrete, 20MPa' (cubic meter, ZA, None),
 'market for concrete, 20MPa' (cubic meter, RNA, None),
 'market for concrete, 20MPa' (cubic meter, RoW, None),
 'market for concrete, 20MPa' (cubic meter, CO, None)]

PE = Peru

ZA = South Africa

RNA = Northern America

RoW = Rest of world

CO = Colombia

In [140]:
fu, data_objs, _ = bd.prepare_lca_inputs({concrete_market: 1}, method=ipcc)

In [141]:
lca = bc.LCA(fu, data_objs=data_objs)

In [142]:
lca.lci()
lca.lcia()

In [144]:
lca.score # global market for "normal" concrete

283.44576866175254

In [150]:
concrete_markets[0]['location']

'PE'

In [151]:
# calculate score for different markets
for market in concrete_markets:
    fu, data_objs, _ = bd.prepare_lca_inputs({market: 1}, method=ipcc)
    lca = bc.LCA(fu, data_objs=data_objs)
    lca.lci()
    lca.lcia()
    print(market['location'], lca.score)

PE 341.7487287312562
ZA 234.01566863447212
RNA 246.39245279678855
RoW 237.44010359161643
CO 250.2176994748129


In [152]:
# but then we could also open these markets up? and see the subactivities for prod of concrete
ba.print_recursive_calculation(concrete_markets[0], ipcc, cutoff=0.001) # CA
# short cut: we assume that the ones contributing most is the ones we need to regionalize
# is there a way to sort this data?

Fraction of score | Absolute score | Amount | Activity
0001 | 341.7 |     1 | 'market for concrete, 20MPa' (cubic meter, PE, None)
  0.0199 | 6.805 |    72 | 'market for transport, freight, lorry >32 metric ton, EURO3' (ton kilo
    0.0199 | 6.805 |    72 | 'transport, freight, lorry >32 metric ton, EURO3' (ton kilometer, RoW,
      0.00291 | 0.9936 | 0.07848 | 'market for road' (meter-year, GLO, None)
      0.00204 | 0.6984 |  1.23 | 'market for diesel, low-sulfur' (kilogram, RoW, None)
  0.00887 | 3.032 |  0.01 | 'concrete production, 20MPa, ready-mix, with cement, limestone 21-35%'
    0.00774 | 2.644 | 3.914 | 'market for cement, limestone 21-35%' (kilogram, PE, None)
      0.00747 | 2.553 | 3.914 | 'cement production, limestone 21-35%' (kilogram, PE, None)
  0.00682 | 2.332 | 0.01048 | 'concrete production, 20MPa, ready-mix, with cement, pozzolana and fly
    0.00569 | 1.945 | 3.237 | 'market for cement, pozzolana and fly ash 36-55%' (kilogram, RoW, None
      0.00164 | 0.5619 | 0

What would happen in ecoinvent if we were to change the concrete markets?

In [ ]:
wood_chipping_act = [act for act in bd.Database(ecoinvent) if 'wood chipping, mobile chipper, at forest road' in act['name'] 
                     and act['location'] == 'RER'][0]

In [153]:
wood_chipping_act # will use this one for now - it doesnt make any sense - we can put in charcrete later

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

Ok so let's go back to the market for conrete, normal and assume that we change one share of the market to our charcrete activity 

In [154]:
ba.print_recursive_calculation(concrete_market, ipcc, cutoff=0.001) # CA
# short cut: we assume that the ones contributing most is the ones we need to regionalize
# is there a way to sort this data?

Fraction of score | Absolute score | Amount | Activity
0001 | 283.4 |     1 | 'market group for concrete, normal' (cubic meter, GLO, None)
  0.0144 | 4.086 | 0.0226 | 'market for concrete, normal' (cubic meter, BR, None)
    0.0144 | 4.086 | 0.0226 | 'concrete, all types to generic market for concrete, normal strength' 
      0.00275 | 0.7798 | 0.003616 | 'market for concrete, 30MPa' (cubic meter, BR, None)
      0.0115 | 3.253 | 0.01876 | 'market for concrete, 25MPa' (cubic meter, BR, None)
  0.00208 | 0.5893 | 0.002355 | 'market for concrete, normal' (cubic meter, CO, None)
    0.00208 | 0.5893 | 0.002355 | 'concrete, all types to generic market for concrete, normal strength' 
      0.00208 | 0.5893 | 0.002355 | 'market for concrete, 20MPa' (cubic meter, CO, None)
  0.0261 |  7.39 | 0.01907 | 'market for concrete, normal' (cubic meter, IN, None)
    0.0261 |  7.39 | 0.01907 | 'concrete, all types to generic market for concrete, normal strength' 
      0.0261 |  7.39 | 0.01907 | 'mark

So basically, this market is in itself 8 other markets: 

CH

BR

CO

IN

PE

RNA

ZA

RoW

Check that they all add up to 1?

In [155]:
sum(exc['amount'] for exc in concrete_market.technosphere()) # does all inputs sum up to one?

1.0000000000000002

idk why it's not exactly one but ok

In [157]:
for exc in concrete_market.technosphere():
    lca.redo_lcia({exc.input.id: 1})
    print(lca.score, exc.input)
    # print the different scores of the different markets i guess

180.79326949989576 'market for concrete, normal' (cubic meter, BR, None)
145.4238269670096 'market for concrete, normal' (cubic meter, CH, None)
250.21769947481295 'market for concrete, normal' (cubic meter, CO, None)
387.578860852761 'market for concrete, normal' (cubic meter, IN, None)
341.74872873125537 'market for concrete, normal' (cubic meter, PE, None)
285.4110569349046 'market for concrete, normal' (cubic meter, RNA, None)
292.9779465046128 'market for concrete, normal' (cubic meter, ZA, None)
283.1982537946509 'market for concrete, normal' (cubic meter, RoW, None)


ok let's start with a weird example: say that we replace all concrete in RNA with charcrete (wood_chipping_act for now)

In [ ]:
concrete_market.copy(Database="make stuff up") # something on gh to show hpw to build a cupply chain in python idk

In [161]:
# ... can we make this into a generic function? like, given an activity, a subactivity and a replacement activity?

# also - can we visualize a before - after?

market_share = 0.0457 # demand of market (market share)
gangnam_style = 1_000_000 # need a large number because somehting about conflicts in id:s in db - dont want to overlap stuff

indices = np.array(
    [
        (gangnam_style, gangnam_style), # Production exchange for new motor ( activity produces itself ) - need to have this to make matrix square
        (concrete_market.id, gangnam_style),  # the whole market
        (bd.get_activity(database=ecoinvent, name='market for concrete, normal', location = 'RNA').id, gangnam_style), # subtract RNA concrete prod
    ] + [
        (node.id, gangnam_style) for node in [wood_chipping_act] # replace with this - something about new dps replacing old ones - could also have the values being added together?
        # see gh bw processing -> policies
        # we have not changes ei - we have added a new activity
        # in these positions - one new row, one new col
    ], dtype=bwp.INDICES_DTYPE
)
data = np.array([
        1,
        1,
        market_share, # old market
    ] + [
        market_share # new market - same amount
    ]
) 
flip = np.array(
    [False, True, False] + [True for _ in [wood_chipping_act]] # First False because ?, True - motor is consumed, False because numbers are negative ... ?, 
) # could alsos set minussign if you prefer but i still wouldnt know where

In [162]:
dp = bwp.create_datapackage()

dp.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    flip_array=flip,
    name="Market without US",
)

In [163]:
_, data_objs, _ = bd.prepare_lca_inputs({concrete_market: 1}, ipcc) # data_objs - still using ei

In [164]:
lca = bc.LCA({motor.id: 1}, data_objs=data_objs + [dp]) # old motor + add [dp] (new motor)
lca.lci()
lca.lcia()
lca.score

283.44576866175174

In [165]:
lca.lcia({gangnam_style: 1}) # new motor
lca.score

283.0965523271845

# Trying something

In [172]:
concrete_RNA = bd.get_activity(database=ecoinvent, name='market for concrete, normal', location = 'RNA')

In [173]:
concrete_RNA

'market for concrete, normal' (cubic meter, RNA, None)

In [185]:
for exc in list(concrete_market.exchanges()):
    print(exc)
    print(exc['name'])
    print(exc.input.id)
    print(concrete_RNA.id)
    if exc.input.id == concrete_RNA.id:
        print('True')
        print(exc['amount'])

Exchange: 1.0 cubic meter 'market group for concrete, normal' (cubic meter, GLO, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
18901
22099
Exchange: 0.0226016949011232 cubic meter 'market for concrete, normal' (cubic meter, BR, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
5266
22099
Exchange: 0.000680925588410356 cubic meter 'market for concrete, normal' (cubic meter, CH, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
18417
22099
Exchange: 0.00235506608812062 cubic meter 'market for concrete, normal' (cubic meter, CO, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
11087
22099
Exchange: 0.01906591647549 cubic meter 'market for concrete, normal' (cubic meter, IN, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
14553
22099
Exchange: 0.0107245780174631 cubic meter 'market for concrete, 

In [196]:
def replace_stuff(main_act, sub_act, repl_act, method):
    '''
    DOCUMENTATION TO BE ADDED
    
    Ideas for development:
    What is really supposed to be returned? Can we return the modefied version of the main act?
    Just first level for now - could we change that? Need to put a maximum "deepness" level?
    '''
    
    
    # first we need the demand of the sub_act in the main act
    for exc in list(main_act.exchanges()):
        if exc.input.id == sub_act.id:
            demand = exc['amount']
    
    gangnam_style = 1_000_000

    indices = np.array(
    [
        (gangnam_style, gangnam_style), # Production exchange for new main_act ( activity produces itself ) - need to have this to make matrix square
        (main_act.id, gangnam_style),  
        (sub_act.id, gangnam_style), # subtract sub_act
    ] + [
        (node.id, gangnam_style) for node in [repl_act] # replace with this - something about new dps replacing old ones - could also have the values being added together?
        # see gh bw processing -> policies
    ], dtype=bwp.INDICES_DTYPE
    )
    
    data = np.array([
            1,
            1,
            demand, # old activity
        ] + [
            demand # new activity - same amount
        ]
    ) 
    flip = np.array(
        [False, True, False] + [True for _ in [repl_act]] # First False because ?, True - motor is consumed, False because numbers are negative ... ?,
        # WAIT I think I get it. This is the data array, where we put False for old act, and True for new act...
        # ...This is probably where we would want to change things to create new market shares...?
    ) # could alsos set minussign if you prefer but i still wouldnt know where
    
    # return(demand)
    
    dp = bwp.create_datapackage()

    dp.add_persistent_vector(
        matrix="technosphere_matrix",
        data_array=data,
        indices_array=indices,
        flip_array=flip,
        name="New technosphere",
    )
    
    _, data_objs, _ = bd.prepare_lca_inputs({main_act: 1}, ipcc) # data_objs - still using ei
    
    lca = bc.LCA({main_act.id: 1}, data_objs=data_objs + [dp]) # old motor + add [dp] (new motor)
    lca.lci()
    lca.lcia()
    lcascore1 = lca.score # first version
    
    lca.lcia({gangnam_style: 1}) # new motor
    lcascore2 = lca.score # substituted sub_act w repl_act
    
    return(lcascore1, lcascore2)

In [197]:
print(replace_stuff(concrete_market, concrete_RNA, wood_chipping_act, ipcc))

(283.44576866175174, 283.0987257563718)


In [198]:
concrete_market

'market group for concrete, normal' (cubic meter, GLO, None)

In [199]:
concrete_RNA

'market for concrete, normal' (cubic meter, RNA, None)

In [200]:
wood_chipping_act

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

In [201]:
ipcc

('IPCC 2013', 'climate change', 'GWP 100a')

now... it would be supercool if we could visualize this...

The amount of each input and the share of the impacts, and the total impacts... before and after...

Let's draw what we want to see

And then - we want to do the cool thing with the ba.print_recursive_calculation(concrete_market, ipcc, cutoff=0.001) - where do we find documentation on that method?

In [263]:
ba.print_recursive_calculation?

Signature:
ba.print_recursive_calculation(
    activity,
    lcia_method,
    amount=1,
    max_level=3,
    cutoff=0.01,
    file_obj=None,
    tab_character='  ',
    use_matrix_values=False,
    _lca_obj=None,
    _total_score=None,
    __level=0,
    __first=True,
)
Docstring:
Traverse a supply chain graph, and calculate the LCA scores of each component. Prints the result with the format:

{tab_character * level }{fraction of total score} ({absolute LCA score for this input} | {amount of input}) {input activity}

Args:
    activity: ``Activity``. The starting point of the supply chain graph.
    lcia_method: tuple. LCIA method to use when traversing supply chain graph.
    amount: int. Amount of ``activity`` to assess.
    max_level: int. Maximum depth to traverse.
    cutoff: float. Fraction of total score to use as cutoff when deciding whether to traverse deeper.
    file_obj: File-like object (supports ``.write``), optional. Output will be written to this object if provided.
   

# Changing stuff

In [292]:
def return_recursive_calculation(
    activity,
    lcia_method,
    amount=1,
    max_level=3,
    cutoff=1e-2,
    file_obj=None,
    tab_character="  ",
    use_matrix_values=False,
    _lca_obj=None,
    _total_score=None,
    __level=0,
    __first=True,
    test_object = {}
):
    """Traverse a supply chain graph, and calculate the LCA scores of each component. Prints the result with the format:

    {tab_character * level }{fraction of total score} ({absolute LCA score for this input} | {amount of input}) {input activity}

    Args:
        activity: ``Activity``. The starting point of the supply chain graph.
        lcia_method: tuple. LCIA method to use when traversing supply chain graph.
        amount: int. Amount of ``activity`` to assess.
        max_level: int. Maximum depth to traverse.
        cutoff: float. Fraction of total score to use as cutoff when deciding whether to traverse deeper.
        file_obj: File-like object (supports ``.write``), optional. Output will be written to this object if provided.
        tab_character: str. Character to use to indicate indentation.
        use_matrix_values: bool. Take exchange values from the matrix instead of the exchange instance ``amount``. Useful for Monte Carlo, but can be incorrect if there is more than one exchange from the same pair of nodes.

    Normally internal args:
        _lca_obj: ``LCA``. Can give an instance of the LCA class (e.g. when doing regionalized or Monte Carlo LCA)
        _total_score: float. Needed if specifying ``_lca_obj``.

    Internal args (used during recursion, do not touch);
        __level: int.
        __first: bool.

    Returns:
        Nothing. Prints to ``sys.stdout`` or ``file_obj``

    """
    activity = bd.get_activity(activity)
    #if file_obj is None:
        #file_obj = sys.stdout # i dont think we need this

    if _lca_obj is None:
        _lca_obj = bc.LCA({activity: amount}, lcia_method)
        _lca_obj.lci()
        _lca_obj.lcia()
        _total_score = _lca_obj.score
    elif _total_score is None:
        raise ValueError
    else:
        _lca_obj.redo_lcia({activity.id: amount})
        if abs(_lca_obj.score) <= abs(_total_score * cutoff):
            return
    if __first:
        print("Fraction of score | Absolute score | Amount | Activity\n")
    message = "{}{:04.3g} | {:5.4n} | {:5.4n} | {:.70}".format(
        tab_character * __level,
        _lca_obj.score / _total_score,
        _lca_obj.score,
        float(amount),
        str(activity),
    )
    print(message + "\n")
    '''
    print("TEST: ",      tab_character * __level, # this is just the indentation, we don't need it
          "Fraction of score? ", 
        _lca_obj.score / _total_score,
          "Absolute score? ",
        _lca_obj.score,
          "Amount? ", 
        float(amount),
          "Activity? ",
        str(activity))
    
    print("LEVEL: ", __level)
    '''
    if __level == 1:
        test_object[str(activity)] = {"Fraction of score? ": _lca_obj.score / _total_score,
          "Absolute score? " : _lca_obj.score,
          "Amount? " : float(amount)}
    
    if __level < max_level:
        prod_exchanges = list(activity.production())
        if not prod_exchanges:
            prod_amount = 1
        elif len(prod_exchanges) > 1:
            warn("Hit multiple production exchanges; aborting in this branch")
            return
        else:
            prod_amount = _lca_obj.technosphere_matrix[
                _lca_obj.dicts.product[prod_exchanges[0].input.id],
                _lca_obj.dicts.activity[prod_exchanges[0].output.id],
            ]

        for exc in activity.technosphere():
            if exc.input.id == exc.output.id:
                continue

            if use_matrix_values:
                sign = (
                    -1
                    if exc.get("type") in ("technosphere", "generic technosphere")
                    else 1
                )
                tm_amount = (
                    _lca_obj.technosphere_matrix[
                        _lca_obj.dicts.product[exc.input.id],
                        _lca_obj.dicts.activity[exc.output.id],
                    ]
                    * sign
                )
            else:
                tm_amount = exc["amount"]

                
            return_recursive_calculation(
                activity=exc.input,
                lcia_method=lcia_method,
                amount=amount * tm_amount / prod_amount,
                max_level=max_level,
                cutoff=cutoff,
                file_obj=file_obj,
                tab_character=tab_character,
                __first=False,
                _lca_obj=_lca_obj,
                _total_score=_total_score,
                __level=__level + 1,
            )
            
    return(test_object)

In [293]:
whats_this = return_recursive_calculation(concrete_market, ipcc, cutoff=0.001)

Fraction of score | Absolute score | Amount | Activity

0001 | 283.4 |     1 | 'market group for concrete, normal' (cubic meter, GLO, None)

  0.0144 | 4.086 | 0.0226 | 'market for concrete, normal' (cubic meter, BR, None)

    0.0144 | 4.086 | 0.0226 | 'concrete, all types to generic market for concrete, normal strength' 

      0.00275 | 0.7798 | 0.003616 | 'market for concrete, 30MPa' (cubic meter, BR, None)

      0.0115 | 3.253 | 0.01876 | 'market for concrete, 25MPa' (cubic meter, BR, None)

  0.00208 | 0.5893 | 0.002355 | 'market for concrete, normal' (cubic meter, CO, None)

    0.00208 | 0.5893 | 0.002355 | 'concrete, all types to generic market for concrete, normal strength' 

      0.00208 | 0.5893 | 0.002355 | 'market for concrete, 20MPa' (cubic meter, CO, None)

  0.0261 |  7.39 | 0.01907 | 'market for concrete, normal' (cubic meter, IN, None)

    0.0261 |  7.39 | 0.01907 | 'concrete, all types to generic market for concrete, normal strength' 

      0.0261 |  7.39 | 0.01

In [294]:
whats_this

{"'market for concrete, normal' (cubic meter, BR, None)": {'Fraction of score? ': 0.014416282651548304,
  'Absolute score? ': 4.086234317413197,
  'Amount? ': 0.0226016949011232},
 "'market for concrete, normal' (cubic meter, CO, None)": {'Fraction of score? ': 0.0020789840026996485,
  'Absolute score? ': 0.5892792186806889,
  'Amount? ': 0.00235506608812062},
 "'market for concrete, normal' (cubic meter, IN, None)": {'Fraction of score? ': 0.026070405720194617,
  'Absolute score? ': 7.389546188684314,
  'Amount? ': 0.01906591647549},
 "'market for concrete, normal' (cubic meter, PE, None)": {'Fraction of score? ': 0.012930554303038166,
  'Absolute score? ': 3.665110903647185,
  'Amount? ': 0.0107245780174631},
 "'market for concrete, normal' (cubic meter, RNA, None)": {'Fraction of score? ': 0.045730467028684975,
  'Absolute score? ': 12.962107378206543,
  'Amount? ': 0.0454155754069573},
 "'market for concrete, normal' (cubic meter, ZA, None)": {'Fraction of score? ': 0.0015833551626

# Trying something - changing market shares

In [ ]:
(node.id, gangnam_style) for node in [repl_act] # this one works...

In [229]:
test=[(exc.input.id) for exc in concrete_market.exchanges()]

In [230]:
test

[18901, 5266, 18417, 11087, 14553, 23135, 22099, 16409, 19248]

In [231]:
test=[(exc.input.id, 1000) for exc in concrete_market.exchanges()]

In [232]:
test

[(18901, 1000),
 (5266, 1000),
 (18417, 1000),
 (11087, 1000),
 (14553, 1000),
 (23135, 1000),
 (22099, 1000),
 (16409, 1000),
 (19248, 1000)]

In [220]:
list(concrete_market.exchanges())

[Exchange: 1.0 cubic meter 'market group for concrete, normal' (cubic meter, GLO, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.0226016949011232 cubic meter 'market for concrete, normal' (cubic meter, BR, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.000680925588410356 cubic meter 'market for concrete, normal' (cubic meter, CH, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.00235506608812062 cubic meter 'market for concrete, normal' (cubic meter, CO, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.01906591647549 cubic meter 'market for concrete, normal' (cubic meter, IN, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.0107245780174631 cubic meter 'market for concrete, normal' (cubic meter, PE, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.0454155754069573 cubic

In [260]:
def change_market_shares(main_act, repl_act, market_share, method):
    
    gangnam_style = 1_000_000

    indices = np.array(
    [
        (gangnam_style, gangnam_style), # Production exchange for new main_act ( activity produces itself ) - need to have this to make matrix square
        (main_act.id, gangnam_style)]
        +
        [
        
        (exc.input.id, 1_000_000) for exc in main_act.exchanges()
        #(sub_act.id, gangnam_style), # subtract sub_act - here we need to add all the sub_acts
    ] + [
        (node.id, gangnam_style) for node in [repl_act] # replace with this - something about new dps replacing old ones - could also have the values being added together?
        # see gh bw processing -> policies
    ], dtype=bwp.INDICES_DTYPE
    )
    
    data = np.array([
            1,
            1,
        ]
        +
        [
        exc['amount']*(1-market_share) for exc in list(main_act.exchanges()) # amound*(1-0.1) = amount*0.9... Try amount*(-0.1) instead! Also try changing flip to False for these
        ]
        +
        [
            market_share # new activity - same amount
        ]
    ) 
    flip = np.array(
        [False, True] + [True for exc in list(main_act.exchanges())] + [True for _ in [repl_act]] # First False because ?, True - motor is consumed, False because numbers are negative ... ?,
        # WAIT I think I get it. This is the data array, where we put False for old act, and True for new act...
        # ...This is probably where we would want to change things to create new market shares...?
    ) # could alsos set minussign if you prefer but i still wouldnt know where
    
    # return(demand)
    
    dp = bwp.create_datapackage()

    dp.add_persistent_vector(
        matrix="technosphere_matrix",
        data_array=data,
        indices_array=indices,
        flip_array=flip,
        name="New technosphere",
    )
    
    _, data_objs, _ = bd.prepare_lca_inputs({main_act: 1}, ipcc) # data_objs - still using ei
    
    
    lca = bc.LCA({main_act.id: 1}, data_objs=data_objs + [dp]) # old motor + add [dp] (new motor)
    lca.lci()
    lca.lcia()
    lcascore1 = lca.score
    
    lca.lcia({gangnam_style: 1}) # new motor
    lcascore2 = lca.score

    
    return(lcascore1, lcascore2)

In [261]:
print(change_market_shares(concrete_market, wood_chipping_act, 0.1, ipcc))

(283.44576866175237, 821.4251004328859)


uhm... so now we get results... how do we check if this is reasonable?

ok - let's break down what we have done

We have taken away 10% of the OG act, and replaces that with 0.1 of the replacement act

So 283 x 0.9 + 0.1 of repl_act should be 5923

let's do an LCA of the repl_act


In [259]:
_, data_objs, _ = bd.prepare_lca_inputs({wood_chipping_act: 0.1}, ipcc) # data_objs - still using ei    
lca = bc.LCA({wood_chipping_act.id: 0.1}, data_objs=data_objs) # old motor + add [dp] (new motor)
lca.lci()
lca.lcia()
lca.score

27.77695616489711

Yeah so that didnt work...

Lets verify w AB

Yes - the LCA score above is correct

So what am I doing wrong? And how do I find what I am doing wrong?

In [ ]:
hydro = [act for act in eidb if "hydro" in act['name'] 
                     and "river" in act['name'] 
                     and "CH" in act['location']
                     and "kilowatt hour" in act['unit']
                     ][0]
hydro

Let's also compare these according to their carbon footprint as measured with the IPCC method we already selected above:

In [ ]:
ipcc_2013_method

#### One at a time approach:

In [ ]:
hydroLCA = bw.LCA({hydro:1}, ipcc_2013_method.name)
hydroLCA.lci()
hydroLCA.lcia()
hydroLCA.score

**Exercise:** Do the LCA for `wind`:

In [ ]:
windLCA = bw.LCA({wind:1}, ipcc_2013_method.name)
windLCA.lci()
windLCA.lcia()
windLCA.score

In [ ]:
#Compare results:
if windLCA.score>hydroLCA.score:
    print("Hydro is preferable")
elif windLCA.score<hydroLCA.score:
    print("Wind is preferable")
else:
    print("Both options have the same climate change indicator result")

Do one "delta" LCA:

In [ ]:
deltaLCA = bw.LCA({wind:1, hydro:-1}, ipcc_2013_method.name)
deltaLCA.lci()
deltaLCA.lcia()
deltaLCA.score

In [ ]:
if deltaLCA.score>0:
    print("Hydro is preferable")
elif deltaLCA.score<0:
    print("Wind is preferable")
else:
    print("Both options have the same climate change indicator result")